In [1]:
import os
import pandas as pd # type: ignore
import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore
from glob import glob
import numpy as np # type: ignore
import cv2 # type: ignore

In [2]:
# 학습 데이터의 경로와 정보를 가진 파일의 경로를 설정
traindata_dir = "./data/train"
traindata_info_file = "./data/train.csv"

In [3]:
os.chdir('/data/ephemeral/home/cv20-proj1/level1-imageclassification-cv-20/suyoung')
train_data = pd.read_csv(traindata_info_file)

In [7]:
train_data.head()

,class_name,image_path,target
0,n01872401,n01872401/sketch_50.JPEG,59
1,n02417914,n02417914/sketch_11.JPEG,202
2,n02106166,n02106166/sketch_3.JPEG,138
3,n04235860,n04235860/sketch_2.JPEG,382
4,n02056570,n02056570/sketch_40.JPEG,80


In [6]:
folders = []
# data 폴더 밑에 canny 폴더를 먼저 만들고 수행 
canny_dir = '/data/ephemeral/home/cv20-proj1/level1-imageclassification-cv-20/suyoung/data/canny'

for i in range(len(train_data)):
    class_name = train_data['class_name'].values[i]
    original_image_path = os.path.join(traindata_dir, train_data['image_path'].values[i])
    file_name = train_data['image_path'].values[i].split('/')[-1]

    # Canny edge detection 
    image = cv2.imread(original_image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edge = cv2.Canny(gray, 100, 200)

    if class_name in folders:
        cv2.imwrite(canny_dir+'/'+train_data['image_path'].values[i], edge)
    else:
        folders.append(class_name)
        os.makedirs(canny_dir+'/'+class_name)
        cv2.imwrite(canny_dir+'/'+train_data['image_path'].values[i], edge)
    
    if i % 100 == 0:
        print(f'{i}th work done')


0th work done
100th work done
200th work done
300th work done
400th work done
500th work done
600th work done
700th work done
800th work done
900th work done
1000th work done
1100th work done
1200th work done
1300th work done
1400th work done
1500th work done
1600th work done
1700th work done
1800th work done
1900th work done
2000th work done
2100th work done
2200th work done
2300th work done
2400th work done
2500th work done
2600th work done
2700th work done
2800th work done
2900th work done
3000th work done
3100th work done
3200th work done
3300th work done
3400th work done
3500th work done
3600th work done
3700th work done
3800th work done
3900th work done
4000th work done
4100th work done
4200th work done
4300th work done
4400th work done
4500th work done
4600th work done
4700th work done
4800th work done
4900th work done
5000th work done
5100th work done
5200th work done
5300th work done
5400th work done
5500th work done
5600th work done
5700th work done
5800th work done
5900th wo

In [ ]:
# 해당 셀은 train.py에서 중간에 삽입
# 각 class별로 8:2의 비율이 되도록 학습과 검증 데이터를 분리.
train_df, val_df = train_test_split(
    train_info, 
    test_size=1-config.TRAIN_RATIO,
    stratify=train_info['target'],
    random_state=20
)

# # 새롭게 생성한 canny edge data를 train_df에 해당되는 것만 이동
source_dir = '/data/ephemeral/home/cv20-proj1/level1-imageclassification-cv-20/suyoung/data/canny'  # 원본 디렉토리 경로
destination_dir = '/data/ephemeral/home/cv20-proj1/level1-imageclassification-cv-20/suyoung/data/train'  # 대상 디렉토리 경로

source_folders = os.listdir(source_dir)
destination_folders = os.listdir(destination_dir)

for i in range(len(destination_folders)):
    if destination_folders[i] in source_folders:
        folder = destination_folders[i]
        image_path_list = train_df[train_df['class_name'] == folder]['image_path'].to_list()
        for j in range(len(image_path_list)):
            source_file_path = os.path.join(source_dir, image_path_list[j])
            file_name = image_path_list[j].split('/')[-1]
            new_name = 'edge_detected_' + file_name
            destination_file_path = os.path.join(destination_dir, folder, new_name)
            shutil.copy2(source_file_path, destination_file_path)

# train_info에 추가된 canny edge data를 덧붙임
class_name_list = []
image_path_list = []
target_list = []

traindata_folders = os.listdir(traindata_dir)

for i in range(len(traindata_folders)):
    if traindata_folders[i] != '.DS_Store':
        class_name = traindata_folders[i]
        class_path = os.path.join(traindata_dir, class_name)
        class_target = train_info[train_info['class_name'] == class_name]['target'].unique().item()

        # 클래스 폴더에서 이미지별로 path 추출
        class_images_list = os.listdir(class_path)
        for j in range(len(class_images_list)):
            class_image = class_images_list[j]
            if class_image.startswith('edge_detected'):
                class_image_path = os.path.join(class_name, class_image)
                class_name_list.append(class_name)
                image_path_list.append(class_image_path)
                target_list.append(class_target)

# 데이터 프레임에 추가하기 
new_data = pd.DataFrame({train_info.columns[0] : class_name_list, 
                         train_info.columns[1] : image_path_list,
                         train_info.columns[2] : target_list})

train_info = pd.concat([train_info, new_data], ignore_index=True)
train_info = train_info.astype({'target' : 'int'})